In [57]:
n = 4 # choose the number of movies for the consideration in the top "n" movies study 

In [58]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt

In [59]:
#df = pd.read_excel("ratings_taha.xlsx")
#df = pd.read_csv("ratings_alame.csv")
df = pd.read_csv("ratings.csv")
df.head()

,Const,Your Rating,Date Rated,Title,URL,Title Type,IMDb Rating,Runtime (mins),Year,Genres,Num Votes,Release Date,Directors
0,tt1001526,6,2016-03-26,Megamind,https://www.imdb.com/title/tt1001526/,movie,7.3,95.0,2010,"Animation, Action, Comedy, Crime, Family, Myst...",266713.0,2010-10-28,Tom McGrath
1,tt10062292,6,2022-12-19,Never Have I Ever,https://www.imdb.com/title/tt10062292/,tvSeries,7.9,30.0,2020,"Comedy, Drama",60236.0,2020-02-27,NaN
2,tt1010048,7,2022-10-08,Slumdog Millionaire,https://www.imdb.com/title/tt1010048/,movie,8.0,120.0,2008,"Crime, Drama, Romance",848648.0,2008-08-30,"Loveleen Tandan, Danny Boyle"
3,tt1013542,3,2022-10-08,The Cheetah Girls: One World,https://www.imdb.com/title/tt1013542/,tvMovie,4.6,90.0,2008,"Comedy, Drama, Family, Musical",4584.0,2008-08-12,Paul Hoen
4,tt1013752,5,2022-10-08,Fast & Furious,https://www.imdb.com/title/tt1013752/,movie,6.5,107.0,2009,"Action, Crime, Thriller",290248.0,2009-03-12,Justin Lin


In [60]:
df = df[["Directors","Your Rating"]]

In [61]:
df = df.dropna()
df. isna(). sum(). sum() 
df = df.reset_index()

In [62]:
for ind in df.index:
    element = df["Directors"][ind]
    directors_list = element.split(",")
    if len(directors_list) > 1:  
        for i in range(len(directors_list)):
            if i>0:
                df.loc[len(df.index)]=df.loc[ind]
                df.iloc[[len(df.index)-1],[1]] = directors_list[i][1:]
            else:
                df.iloc[[ind],[1]] = directors_list[0]

In [63]:
D = {}

for ind in df.index:
    element = df["Directors"][ind]
    rating = df["Your Rating"][ind]
    if element in D:
        D[element].append(rating)
    else:
        D[element]=[rating]
        

In [64]:
new_df = pd.DataFrame(columns=['Director', 'Number of Movies', 'Average Rating'])

for element in D:
    count = len(D[element])
    average = sum(D[element])/count
    new_df = new_df.append({'Director': element, 'Number of Movies': count, 'Average Rating': average}, ignore_index=True) 

In [65]:
new_df_count = new_df.sort_values(by=['Number of Movies',"Average Rating"],ascending=False)

new_df_average = new_df.sort_values(by=['Average Rating',"Number of Movies"],ascending=False)


In [66]:
new_df_2_movies = new_df_count[new_df_count['Number of Movies'] == 2]
new_df_2_movies = new_df_2_movies.reset_index()
new_df_2_movies = new_df_2_movies.drop(columns=['index'])
new_df_2_movies.index += 1

new_df_count = new_df_count[new_df_count['Number of Movies'] >= 3]
new_df_count = new_df_count.reset_index()
new_df_count = new_df_count.drop(columns=['index'])
new_df_count.index += 1

new_df_average = new_df_average[new_df_average['Number of Movies'] >= 3]
new_df_average = new_df_average.reset_index()
new_df_average = new_df_average.drop(columns=['index'])
new_df_average.index += 1

In [67]:
def max_n(L,n):
    # returns the largest n elements of a list 
    max_list = []
    for i in range(n):
        x = max(L)
        max_list.append(x)
        L.remove(x)
    return max_list 

In [68]:
D_tilda = {} # modified dictionary to only consider the first n movies 

for element in D:
    if len(D[element]) >= n:
        D_tilda[element] = max_n(D[element],n)


In [69]:
max_df = pd.DataFrame(columns=['Director', "Top "+str(n)+" Movies"])

for element in D_tilda:
    average = sum(D_tilda[element])/n
    max_df = max_df.append({'Director': element, "Top "+str(n)+" Movies": average}, ignore_index=True) 

In [70]:
max_df = max_df.sort_values(by=["Top "+str(n)+" Movies"],ascending=False)
max_df= max_df.reset_index()
max_df = max_df.drop(columns=['index'])
max_df.index += 1
max_df["Top "+str(n)+" Movies"] = max_df["Top "+str(n)+" Movies"].round(2)

In [71]:
# A more fair sorting for like ratings:

max_df = max_df.set_index("Director")
new_df_average = new_df_average.set_index("Director")
max_df = pd.merge(max_df, new_df_average, left_index=True, right_index=True) 

max_df = max_df.sort_values(by=["Top "+str(n)+" Movies","Average Rating"],ascending=False)

max_df= max_df.reset_index()
max_df.index += 1
max_df = max_df.drop(columns=['Number of Movies','Average Rating'])

new_df_average = new_df_average.reset_index()
new_df_average.index += 1

In [72]:
new_df_count

,Director,Number of Movies,Average Rating
1,Wes Anderson,13,6.000000
2,Quentin Tarantino,11,6.000000
3,Martin Scorsese,11,5.454545
4,Christopher Nolan,11,5.454545
5,Steven Soderbergh,11,5.000000
6,David O. Russell,10,6.100000
7,Edgar Wright,9,5.888889
8,Paul Feig,9,5.333333
9,Sam Raimi,8,5.500000
10,Alejandro G. Iñárritu,7,6.857143


In [73]:
new_df_average

,Director,Number of Movies,Average Rating
1,Martin McDonagh,5,8.000000
2,Alejandro G. Iñárritu,7,6.857143
3,David O. Russell,10,6.100000
4,Wes Anderson,13,6.000000
5,Quentin Tarantino,11,6.000000
6,Ruben Fleischer,6,6.000000
7,Danny Boyle,4,6.000000
8,Christopher Miller,4,6.000000
9,Phil Lord,4,6.000000
10,James Cameron,3,6.000000


In [74]:
new_df_2_movies

,Director,Number of Movies,Average Rating
1,Ridley Scott,2,9.0
2,Frank Darabont,2,8.5
3,Rian Johnson,2,8.5
4,Bennett Miller,2,8.0
5,Mel Gibson,2,8.0
...,...,...,...
59,Raja Gosnell,2,2.0
60,Dan Kwan,2,2.0
61,Daniel Scheinert,2,2.0
62,James Wong,2,1.5


In [75]:
max_df

,Director,Top 4 Movies
1,David O. Russell,8.50
2,Martin McDonagh,8.25
3,Alejandro G. Iñárritu,8.25
4,Quentin Tarantino,8.25
5,Wes Anderson,7.50
6,Edgar Wright,7.50
7,Steven Soderbergh,7.00
8,Ruben Fleischer,6.75
9,Sam Raimi,6.75
10,Paul Feig,6.75
